In [1]:
import pandas as pd
import datetime
import numpy as np
import re
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
from hashlib import sha256

%matplotlib inline

In [2]:
pd.options.display.max_columns=1000
mlp.rcParams["figure.dpi"] = 200
mlp.rcParams["figure.facecolor"] = "white"

In [3]:

vinculacion = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018.xlsx", 4)
psico = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018.xlsx", 8)
its = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018.xlsx", 6)
aa = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018.xlsx", 7)

In [9]:
pb_tvc = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018.xlsx", 5)

In [5]:
# Unique IDs begin with a letter indicating either Male, Female or Trans. 
# Continue with 6 numbers indicating date of birth. Are followed by 
# 2 unknown numbers and 3 numbers standing for the municipality code, 
# then 4 or 3 letters that stand for the initials of the patient. 
# Finally two numbers of unknown meaning. Must convert dates to age,
# obtain sex column and unique hash

In [6]:
def FuncTry(f):
    def wrapper(data):
        try:
            return f(data)
        except:
            print("Bad data:", data)
            return np.NaN
    return wrapper

def parseDate(input_str, format_ = "%d%m%y"):
    try:
        return datetime.datetime.strptime(input_str[1:7], "%d%m%y")
    except:
        print("Bad date", input_str)
        return pd.NaT

muni_expr = re.compile(r"(\d{4})[A-Z]+")
def extractMunicode(input_str):
    try:
        return int(muni_expr.search(input_str).group(1))
    except:
        print("Bad municode:", input_str)
        return 0
    
def getAgeFuncGen(cui_field):
    def getAge(row):
        try:
            return np.floor((row.Date - parseDate(row[cui_field]).date()).days/365.25)
        except:
            print("Bad dates:", row.Date, row[cui_field])
            return np.NaN
    return getAge
        
def processDatabase(df, cui_field = "codigounico"):
    df["Gender"]  = df[cui_field].map(lambda x: x[0:1] if type(x) == str else np.NaN)
    df["Age"]     = df.apply(getAgeFuncGen(cui_field), axis=1)
    df["municode"]= df[cui_field].map(extractMunicode)
    df[cui_field]     = hashField(df, cui_field)
    
def hashField(df, hash_field):
    df[hash_field]     = df[hash_field].map(FuncTry(lambda x: sha256(hash_field.encode()).hexdigest()))

In [10]:
pb_tvc["Date"]    = pd.to_datetime(pb_tvc.fechareal, format = "%d/%m/%Y").map(lambda x: x.date())
processDatabase(pb_tvc)
hashField(pb_tvc, "responsable")
hashField(pb_tvc, "educador")

Bad date T01258911801BEMO00
Bad date M113/012001MASO00
Bad date M01179310101JUGA00
Bad date T07259310101KEBR00
Bad date M09169810117JOSA00
Bad date M01/19810101FRMA00
Bad date M09179810706FRMA00


PermissionError: [Errno 13] Permission denied: '../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - PB_TVC.csv'

In [11]:
pb_tvc.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - PB_TVC.csv")

In [12]:
its["Date"]    = its.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
#its.iloc[6958]
its.Date.head(2)

0    2018-01-01
1    2018-01-01
Name: Date, dtype: object

In [13]:
#np.floor((its.Date - 
r = its.iloc[1]
(r.Date - parseDate(r["codigounico"]).date()).days/365.25

38.04517453798768

In [14]:
processDatabase(its)
hashField(its, "responsable")
its.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - ITS.csv")

Bad date M01/09912214ELRE00
Bad date M01/09912214ELRE00


In [15]:
aa["Date"]    = aa.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
aa.Date.head(2)

0    2018-01-01
1    2018-01-01
Name: Date, dtype: object

In [16]:
aa.codigounico.map(lambda x: type(x)).value_counts()

<class 'str'>      17641
<class 'float'>        1
Name: codigounico, dtype: int64

In [17]:
processDatabase(aa)
hashField(aa, "responsable")
aa.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - AA.csv")

Bad date M01/09912214ELRE00
Bad date nan
Bad date HSH MAYORES DE 18 AÃ‘OS
Bad date TRANS TRABAJADORAS SEXUALES
Bad date TRANS MAYORES DE 18 AÃ‘OS
Bad date M01/09912214ELRE00
Bad date TRANS TRABAJADORAS SEXUALES
Bad date TRANS MAYORES DE 18 AÃ‘OS
Bad municode: nan
Bad municode: HSH MAYORES DE 18 AÃ‘OS
Bad municode: TRANS TRABAJADORAS SEXUALES
Bad municode: TRANS MAYORES DE 18 AÃ‘OS
Bad municode: TRANS TRABAJADORAS SEXUALES
Bad municode: TRANS MAYORES DE 18 AÃ‘OS


In [18]:
psico["Date"] = psico.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
psico.Date.head(2)

0    2018-04-01
1    2018-04-01
Name: Date, dtype: object

In [19]:
processDatabase(psico)
hashField(psico, "responsable")
psico.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - Psico.csv")

In [20]:
vinculacion["Date"] = vinculacion.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
vinculacion.Date.head(2)

0    2018-02-01
1    2018-03-01
Name: Date, dtype: object

In [21]:
processDatabase(vinculacion)
hashField(vinculacion, "responsable")
vinculacion.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - Vinculacion.csv")

# Some exploring:


In [24]:
vinculacion.Date.map(lambda x: x.year*100+x.month).value_counts().sort_index()

201801    23
201802    37
201803    24
201804    61
201805    65
201806    51
201807    59
201808    79
201809    72
201810    56
201811    54
Name: Date, dtype: int64